In [1]:
!pip install xarray
!pip install netCDF4
!pip install h5netcdf
!pip install rasterio

In [2]:
import google.cloud.storage
import io
import numpy as np
import matplotlib.pyplot as plt
import xarray
import rasterio
from natsort import natsorted 
import tempfile
from rasterio.io import MemoryFile
from rasterio.plot import show
import tensorflow as tf
import pickle

In [3]:
client = google.cloud.storage.Client()

In [4]:
bucket = client.get_bucket('era-ml-upressing')

In [5]:
#select period
selection = np.s_[1460:]  #select 1980, 1980-01-02 to 1980-12-31 leap year

#ERA-20C
files_folder_era20C= !gsutil ls -r 'gs://era-ml-upressing/Predict'
files_era20C_list = natsorted(files_folder_era20C[2:])
files_era20C= [file[22:] for file in files_era20C_list[selection]]

#ERA5
files_folder_era5= !gsutil ls -r 'gs://era-ml-upressing/Test'
files_era5_list = natsorted(files_folder_era5[2:])
files_era5= [file[22:] for file in files_era5_list[:]]


In [47]:
from rasterio.transform import Affine
from rasterio.enums import Resampling
#to calculate mean RMSE of 2000
RMSEint=[]
RMSEdown=[]
for i in range(364):
    day_nr=i
    day= np.s_[day_nr]
    fileERA20C = files_era20C[day]
    fileERA5 = files_era5[day]
    blob = bucket.get_blob(fileERA20C)
    blob2 = bucket.get_blob(fileERA5)                        
    stream = io.BytesIO(blob.download_as_string())  
    stream2 = io.BytesIO(blob2.download_as_string())  
    with MemoryFile(stream) as memfile:                          
        with memfile.open() as dataset:                          #open file, dataset contains all data
            ERA20c = xarray.open_rasterio(dataset)
            arr_256 = dataset.read(out_shape=(dataset.count, 256,256), resampling=Resampling.bilinear)
    with MemoryFile(stream2) as memfile2:                          
        with memfile2.open() as dataset2: 
            ERA5 = xarray.open_rasterio(dataset2)
            ERA5_flat = np.ndarray.flatten(ERA5[4].data)
    with xarray.open_dataset('DownscaledERA20C-2000-2001.nc') as downscaled:
        downscaled_day = downscaled.isel(time=day_nr).squeeze()
        downscaled_arr = xarray.Dataset.to_array(downscaled_day)
        downscaled_flat = np.ndarray.flatten(downscaled_arr.data)
    dif = ERA5_flat - downscaled_flat 
    dif_map = dif.reshape(256,256)
    dif2 = ERA5_flat - np.ndarray.flatten(arr_256)
    dif2map = dif2.reshape(256,256)
    RMSEdown.append((np.sqrt(dif_map **2)).mean()*1000)
    RMSEint.append((np.sqrt(dif2map **2)).mean()*1000)



In [6]:
from rasterio.transform import Affine
from rasterio.enums import Resampling
# for plotting, choose day number
day_nr=3
day= np.s_[day_nr]
fileERA20C = files_era20C[day]
fileERA5 = files_era5[day]
blob = bucket.get_blob(fileERA20C)
blob2 = bucket.get_blob(fileERA5)                        
stream = io.BytesIO(blob.download_as_string())  
stream2 = io.BytesIO(blob2.download_as_string())  
with MemoryFile(stream) as memfile:                          
    with memfile.open() as dataset:                          #open file, dataset contains all data
        ERA20c = xarray.open_rasterio(dataset)
        arr_256 = dataset.read(out_shape=(dataset.count, 256,256), resampling=Resampling.bilinear)
with MemoryFile(stream2) as memfile2:                          
    with memfile2.open() as dataset2: 
        ERA5 = xarray.open_rasterio(dataset2)
        ERA5_flat = np.ndarray.flatten(ERA5[4].data)
with xarray.open_dataset('DownscaledERA20C-2000-2001.nc') as downscaled:
    downscaled_day = downscaled.isel(time=day_nr).squeeze()
    downscaled_arr = xarray.Dataset.to_array(downscaled_day)
    downscaled_flat = np.ndarray.flatten(downscaled_arr.data)
dif = ERA5_flat - downscaled_flat 
dif_map = dif.reshape(256,256)
dif2 = ERA5_flat - np.ndarray.flatten(arr_256)
dif2map = dif2.reshape(256,256)



In [ ]:
#plot day that is specified in the cell above, plot low res, high res, downscaled, interpolated and difference map
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import davos
!pip install cmocean
import cmocean
norm= colors.PowerNorm(gamma=0.5)
plt.rcParams['font.size'] = 16
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 16
fig, axes = plt.subplots(1,5, figsize=(28,4))


show(ERA20c*1000, ax=axes[0], title='ERA-20C', transform=ERA20c.transform, vmin=0, vmax=20,  cmap=davos.davos_map, interpolation=None) 
show(downscaled_arr*1000, ax=axes[1], title='Downscaled RMSE {:.2f}'.format((np.sqrt(dif_map **2)).mean()*1000),  transform = ERA5[4].transform, vmin=0, vmax=20,  cmap=davos.davos_map, interpolation=None) 
show(arr_256*1000, ax=axes[2], title='Interpolated RMSE {:.2f}'.format((np.sqrt(dif2map **2)).mean()*1000), transform=ERA5[4].transform, vmin=0, vmax=20,  cmap=davos.davos_map, interpolation=None)
show(ERA5[4]*1000, ax=axes[3], title='ERA5', transform = ERA5[4].transform, vmin=0, vmax=20,  cmap=davos.davos_map, interpolation=None)
show(dif_map*1000, ax=axes[4], title='Difference', transform = ERA5[4].transform, vmin=-20, vmax=20, cmap=cmocean.cm.balance)   

for ax in axes[:-1]:
    im = ax.get_images()[0]
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_xticks(ticks=[-20, 0, 20, 40])
    ax.set_yticks(ticks=[20, 40, 60, 80])
    plt.colorbar(im, ax=ax, orientation='vertical', ticks=[0, 2, 5, 10,15,20], format='%.0f', fraction=0.046, pad=0.04)
im= axes[-1].get_images()[0] 

axes[-1].set_xlabel('Longitude')
axes[-1].set_ylabel('Latitude')
plt.colorbar(im, ax=axes[-1], orientation='vertical', fraction=0.046, pad=0.04, format='%.0f')
plt.savefig('fig.png')

In [29]:
########  calculate MS-SSIM for report ###############
from rasterio.transform import Affine
from rasterio.enums import Resampling
ssim_down=[]
ssim_int=[]
for i in range(364):
    day_nr=i
    day= np.s_[day_nr]
    fileERA20C = files_era20C[day]
    fileERA5 = files_era5[day]
    blob = bucket.get_blob(fileERA20C)
    blob2 = bucket.get_blob(fileERA5)                        
    stream = io.BytesIO(blob.download_as_string())  
    stream2 = io.BytesIO(blob2.download_as_string())  
    with MemoryFile(stream) as memfile:                          
        with memfile.open() as dataset:                          #open file, dataset contains all data
            ERA20c = xarray.open_rasterio(dataset)
            arr_256 = dataset.read(out_shape=(dataset.count, 256,256), resampling=Resampling.bilinear)
    with MemoryFile(stream2) as memfile2:                          
        with memfile2.open() as dataset2: 
            ERA5 = xarray.open_rasterio(dataset2)
    with xarray.open_dataset('DownscaledERA20C-2000-2001.nc') as downscaled:
        downscaled_day = downscaled.isel(time=day_nr).squeeze()
        downscaled_arr = xarray.Dataset.to_array(downscaled_day)

    im1=tf.convert_to_tensor(ERA5[4].data.reshape(1,256,256,1), dtype=tf.float32)
    max_val = np.max(np.reshape(im1, [-1]))
    im2=tf.convert_to_tensor(downscaled_arr.values.reshape(1,256,256,1), dtype=tf.float32)
    im3=tf.convert_to_tensor(arr_256.reshape(1,256,256,1), dtype=tf.float32)
    ssim_down.append(tf.image.ssim_multiscale(im1, im2, max_val=max_val, filter_size=8).numpy())
    ssim_int.append(tf.image.ssim_multiscale(im1, im3, max_val=max_val, filter_size=8).numpy())
list_down= np.concatenate(ssim_down, axis=0)
list_int = np.concatenate(ssim_int, axis=0)